### Import packages

In [1]:
import numpy as np
import pandas as pd
import requests as requests
from io import StringIO
import csv
import os
from datetime import datetime
import pytz
from time import time
import matplotlib.pyplot as plt

### Prepering raw dataset for analysis
#### All information about dataset you can find in [README.MD](https://github.com/MSI17819/Berlin_bike_theft_prediction/blob/main/README.md) file

#### Create empty dataframe

In [2]:
df = pd.DataFrame()

#### Data pulling function from url adress, where we can find data about Berlin bikes thief record from 01/01/2021-up to today.
#### The data are updated everyday or at least once a two/three day, so when we run function the data are expanding on new records.

In [3]:
def data_pulling_function():
    global df
    url = "https://www.internetwache-polizei-berlin.de/vdb/Fahrraddiebstahl.csv"
    response = requests.get(url)
    df_raw = pd.read_csv(StringIO(response.text))
    local_time = datetime.now(pytz.timezone('Europe/Warsaw'))
    df = df.append(df_raw)
    pd.set_option('display.max_columns', None)
    print('Data update - {}'.format(local_time.strftime("%Y-%m-%d %H:%M:%S")))
    return df

data_pulling_function()

Data update - 2023-01-04 23:16:44


C:\Users\dell\AppData\Local\Temp/ipykernel_2064/789137709.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_raw)


,ANGELEGT_AM,TATZEIT_ANFANG_DATUM,TATZEIT_ANFANG_STUNDE,TATZEIT_ENDE_DATUM,TATZEIT_ENDE_STUNDE,LOR,SCHADENSHOEHE,VERSUCH,ART_DES_FAHRRADS,DELIKT,ERFASSUNGSGRUND
0,03.01.2023,01.01.2023,16,02.01.2023,9,2400522,700,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
1,03.01.2023,02.01.2023,18,02.01.2023,22,4300621,1950,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2,03.01.2023,02.01.2023,9,03.01.2023,9,6100101,449,Nein,Fahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
3,03.01.2023,31.12.2022,16,02.01.2023,8,2500835,1420,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
4,03.01.2023,03.01.2023,12,03.01.2023,14,1400942,5150,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
...,...,...,...,...,...,...,...,...,...,...,...
22418,02.01.2022,01.01.2022,19,02.01.2022,18,9501940,449,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
22419,01.01.2022,01.01.2022,20,01.01.2022,20,1300730,150,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
22420,01.01.2022,01.01.2022,4,01.01.2022,8,2100106,190,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
22421,01.01.2022,01.01.2022,14,01.01.2022,14,3601142,1000,Nein,Herrenfahrrad,Fahrraddiebstahl,Einfacher Diebstahl von Fahrrädern


### Preprocesing raw data
- Change columns name 
-  Merging data category

#### Change columns names

In [4]:
df.columns = ["Record_date", "Stealing_date", "Stealing_hour", "Report_stealing_date",
              "Report_stealing_hour", "Berlin_code_area", "Bike_value", "Unsuccesful_attempt",
              "Bike_type", "Crime_type", "Crime_reason"]

#### Change data category in *Unsuccesful_attempt* for three categories

In [5]:
df["Unsuccesful_attempt"].replace({'Nein' : 'No', 'Ja' : 'Yes', 'Unbekannt' : 'Unknown'}, inplace=True)

#### Merge two category *Fahrrad* and *diverse Fahrrader* from *Bike_type* column as one category *Bike*

In [6]:
df["Bike_type"].replace({"Herrenfahrrad" : "Men", "Damenfahrrad" : "Women", "Fahrrad" : "Bike", 
                         "Mountainbike" : "Mountain", "Kinderfahrrad" : "Child", 
                         "diverse Fahrräder" : "Bike", "Rennrad" : "Racing", "Lastenfahrrad" : "Cargo"}, inplace=True)

#### Change column *Crime_type* vales

In [7]:
df["Crime_type"].replace(to_replace=["Fahrraddiebstahl", "Keller- und Bodeneinbruch"], 
                         value=["Theft", "Break-in"], inplace=True)

#### Change type of category *Crime_reason*
#### Category *aggrevated theft* = theft with break-in, theft with robbery (aggresor has to use force) 

In [8]:
df["Crime_reason"].replace(to_replace=["Sonstiger schwerer Diebstahl von Fahrrädern", 
                                       "Einfacher Diebstahl von Fahrrädern",
                                       "Sonstiger schwerer Diebstahl in/aus Keller/Boden von Fahrrädern", 
                                       "Einfacher Diebstahl aus Keller/Boden von Fahrrädern"],
                           value=["Aggravated theft", "Theft", "Aggravated theft from basement/floor", 
                                  "Theft from basement/floor"], inplace=True)

#### Changing column *Record_date* type from object to datetime format

In [9]:
df['Record_date'] = pd.to_datetime(df['Record_date'], dayfirst= True)

#### Create support column *date2* for next step spliting

In [10]:
df['date2'] = pd.to_datetime(df['Record_date'], dayfirst= True)

#### Create three new columns with: day, month and year

In [11]:
df['Record_day'] = df['date2'].dt.day
                                    
df['Record_month'] = df['date2'].dt.month

df['Record_year'] = df['date2'].dt.year

#### Moving three new columns for secound, third and fourth position in DataFrame

In [12]:
col_to_move = df.pop('Record_day')
col_to_move_1 = df.pop('Record_month')
col_to_move_2 = df.pop('Record_year')

df.insert(1, 'Record_day', col_to_move)
df.insert(2, 'Record_month', col_to_move_1)
df.insert(3, 'Record_year', col_to_move_2)

#### Changing *Stealing_date* type from object to datetime format

In [13]:
df['Stealing_date'] = pd.to_datetime(df['Stealing_date'], dayfirst= True)

#### Changing *Stealing_date* type from object to datetime format

In [14]:
df['Report_stealing_date'] = pd.to_datetime(df['Report_stealing_date'], dayfirst= True)

#### Change format into two columns (*Report_stealing_hour'*, *Stealing_hour*) from object to datetime hour format

In [15]:
df['Stealing_hour'] = pd.to_datetime(df['Stealing_hour'], format='%H').dt.hour

df['Report_stealing_hour'] = pd.to_datetime(df['Report_stealing_hour'],  format='%H').dt.hour 

#### Drop support colum *date2* from DataFrame

In [16]:
df.drop(['date2'], axis=1, inplace=True)

#### Display new DataFrame

In [17]:
df

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason
0,2023-01-03,3,1,2023,2023-01-01,16,2023-01-02,9,2400522,700,No,Women,Theft,Aggravated theft
1,2023-01-03,3,1,2023,2023-01-02,18,2023-01-02,22,4300621,1950,No,Women,Theft,Aggravated theft
2,2023-01-03,3,1,2023,2023-01-02,9,2023-01-03,9,6100101,449,No,Bike,Theft,Aggravated theft
3,2023-01-03,3,1,2023,2022-12-31,16,2023-01-02,8,2500835,1420,No,Women,Theft,Aggravated theft
4,2023-01-03,3,1,2023,2023-01-03,12,2023-01-03,14,1400942,5150,No,Men,Theft,Aggravated theft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22418,2022-01-02,2,1,2022,2022-01-01,19,2022-01-02,18,9501940,449,No,Men,Theft,Aggravated theft
22419,2022-01-01,1,1,2022,2022-01-01,20,2022-01-01,20,1300730,150,No,Men,Theft,Aggravated theft
22420,2022-01-01,1,1,2022,2022-01-01,4,2022-01-01,8,2100106,190,No,Men,Theft,Aggravated theft
22421,2022-01-01,1,1,2022,2022-01-01,14,2022-01-01,14,3601142,1000,No,Men,Theft,Theft


#### Load csv file with LOR (Lebensweltlich orientierte Räume (LOR) = Spatial Planning Unit for Berlin) 
#### The dataset was prepared in advance. File contain columns: 
- *Berlin_code_area*
- *Administrative_district*
- *District* 
- *Quater*
- *Street_name* 

#### Load data about LOR (Lebensweltlich orientierte Räume (LOR) = Spatial Planning Unit

In [18]:
url = r"https://raw.githubusercontent.com/MSI17819/Berlin_bike_theft_prediction/main/Berlin_LOR_code.csv"

df_region = pd.read_csv(url, sep=';', encoding="ISO-8859-2")

df_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Berlin_code_area         542 non-null    int64 
 1   Administrative_district  542 non-null    object
 2   District                 542 non-null    object
 3   Quarter                  542 non-null    object
 4   Street_name              542 non-null    object
dtypes: int64(1), object(4)
memory usage: 21.3+ KB


#### Join two data frame (*left join*) with prepared data and LOR dataset

In [19]:
df_merge = pd.merge(df, df_region, how='left', on='Berlin_code_area')

#### Inspection DataFrame after merging two datasets 

In [20]:
df_merge

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name
0,2023-01-03,3,1,2023,2023-01-01,16,2023-01-02,9,2400522,700,No,Women,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Friedrichshain West,Karl-Marx-Allee Nord,Richard-Sorge-Viertel
1,2023-01-03,3,1,2023,2023-01-02,18,2023-01-02,22,4300621,1950,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Otto-Suhr-Allee/Kantstraße,Richard-Wagner-Straße
2,2023-01-03,3,1,2023,2023-01-02,9,2023-01-03,9,6100101,449,No,Bike,Theft,Aggravated theft,Steglitz-Zehlendorf,Steglitz,Schloßstraße,Fichtenberg
3,2023-01-03,3,1,2023,2022-12-31,16,2023-01-02,8,2500835,1420,No,Women,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Friedrichshain Ost,Frankfurter Allee Süd FK,Stralauer Kiez
4,2023-01-03,3,1,2023,2023-01-03,12,2023-01-03,14,1400942,5150,No,Men,Theft,Aggravated theft,Mitte,Wedding,Parkviertel,Westliche Müllerstraße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22418,2022-01-02,2,1,2022,2022-01-01,19,2022-01-02,18,9501940,449,No,Men,Theft,Aggravated theft,Treptow-Köpenick,Treptow-Köpenick 5,Dammvorstadt,Güterbahnhof Köpenick
22419,2022-01-01,1,1,2022,2022-01-01,20,2022-01-01,20,1300730,150,No,Men,Theft,Aggravated theft,Mitte,Gesundbrunnen,Osloer Straße,Drontheimer Straße
22420,2022-01-01,1,1,2022,2022-01-01,4,2022-01-01,8,2100106,190,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Nord,Südliche Friedrichstadt,Wassertorplatz
22421,2022-01-01,1,1,2022,2022-01-01,14,2022-01-01,14,3601142,1000,No,Men,Theft,Theft,Pankow,Nördlicher Prenzlauer Berg,Prenzlauer Berg Nordwest,Falkplatz


### Cleaning data

#### Duplicates inspection

In [21]:
df_merge.duplicated().value_counts()

False    22379
True        44
dtype: int64

#### Deleting duplicates

In [22]:
 df_merge.drop_duplicates(keep='first', inplace=True)

#### DataFrame inspection after deleting duplicated rows

In [23]:
df_merge.duplicated().value_counts()

False    22379
dtype: int64

#### DataFrame inspection about missing values

In [24]:
df_merge.isna().sum()

Record_date                0
Record_day                 0
Record_month               0
Record_year                0
Stealing_date              0
Stealing_hour              0
Report_stealing_date       0
Report_stealing_hour       0
Berlin_code_area           0
Bike_value                 0
Unsuccesful_attempt        0
Bike_type                  0
Crime_type                 0
Crime_reason               0
Administrative_district    0
District                   0
Quarter                    0
Street_name                0
dtype: int64

#### Inspection about category *Unsuccesfull_attempt*

In [25]:
df_merge['Unsuccesful_attempt'].value_counts()

No         22309
Yes           62
Unknown        8
Name: Unsuccesful_attempt, dtype: int64

#### Filter data only for those where thiefs happen and unhappen (*Unsuccesfull_attempt == No, Unsuccesfull_attempt == Yes*), exclude data with *Unknow* category.

In [26]:
df_merge = df_merge[(df_merge['Unsuccesful_attempt'] == 'No') | (df_merge['Unsuccesful_attempt'] == 'Yes')]

#### Inspection DataFrame after droping record with *Unknown* category

In [27]:
df_merge['Unsuccesful_attempt'].value_counts()

No     22309
Yes       62
Name: Unsuccesful_attempt, dtype: int64

#### Reset index and save changes

In [28]:
df_merge.reset_index(drop=True, inplace=True)

#### Dispay DataFrame

In [29]:
df_merge

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name
0,2023-01-03,3,1,2023,2023-01-01,16,2023-01-02,9,2400522,700,No,Women,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Friedrichshain West,Karl-Marx-Allee Nord,Richard-Sorge-Viertel
1,2023-01-03,3,1,2023,2023-01-02,18,2023-01-02,22,4300621,1950,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg Zentrum,Otto-Suhr-Allee/Kantstraße,Richard-Wagner-Straße
2,2023-01-03,3,1,2023,2023-01-02,9,2023-01-03,9,6100101,449,No,Bike,Theft,Aggravated theft,Steglitz-Zehlendorf,Steglitz,Schloßstraße,Fichtenberg
3,2023-01-03,3,1,2023,2022-12-31,16,2023-01-02,8,2500835,1420,No,Women,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Friedrichshain Ost,Frankfurter Allee Süd FK,Stralauer Kiez
4,2023-01-03,3,1,2023,2023-01-03,12,2023-01-03,14,1400942,5150,No,Men,Theft,Aggravated theft,Mitte,Wedding,Parkviertel,Westliche Müllerstraße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22366,2022-01-02,2,1,2022,2022-01-01,19,2022-01-02,18,9501940,449,No,Men,Theft,Aggravated theft,Treptow-Köpenick,Treptow-Köpenick 5,Dammvorstadt,Güterbahnhof Köpenick
22367,2022-01-01,1,1,2022,2022-01-01,20,2022-01-01,20,1300730,150,No,Men,Theft,Aggravated theft,Mitte,Gesundbrunnen,Osloer Straße,Drontheimer Straße
22368,2022-01-01,1,1,2022,2022-01-01,4,2022-01-01,8,2100106,190,No,Men,Theft,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Nord,Südliche Friedrichstadt,Wassertorplatz
22369,2022-01-01,1,1,2022,2022-01-01,14,2022-01-01,14,3601142,1000,No,Men,Theft,Theft,Pankow,Nördlicher Prenzlauer Berg,Prenzlauer Berg Nordwest,Falkplatz


In [30]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22371 entries, 0 to 22370
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Record_date              22371 non-null  datetime64[ns]
 1   Record_day               22371 non-null  int64         
 2   Record_month             22371 non-null  int64         
 3   Record_year              22371 non-null  int64         
 4   Stealing_date            22371 non-null  datetime64[ns]
 5   Stealing_hour            22371 non-null  int64         
 6   Report_stealing_date     22371 non-null  datetime64[ns]
 7   Report_stealing_hour     22371 non-null  int64         
 8   Berlin_code_area         22371 non-null  int64         
 9   Bike_value               22371 non-null  int64         
 10  Unsuccesful_attempt      22371 non-null  object        
 11  Bike_type                22371 non-null  object        
 12  Crime_type               22371 n

#### Export DataFrame to csv file for future prediction analysis

In [31]:
df_merge.to_csv(r'C:\Users\dell\Desktop\Project\Berlin_bike_theft_prediction\Berlin_bike_thief_file_for_analysis.csv', 
                sep=',', encoding='utf-8', header=True, index=False)